<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/inspect_train_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine confidence threshold for Image Type Classification Models 
---
*Last Updated 30 October 2020*   
Choose which trained model and confidence threshold values to use for classifying flowers/fruits from EOL images. Threshold values should be chosen that maximize coverage and minimize error.

First, choose the 2 best models trained in [image_type_train.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/image_type_preprocessing.ipynb). Then, run this notebook. 

1) Save model predictions and confidence values for 1500 images per class  (Map, Phylogeny, Illustration, Herbarium Sheet, Null) for each model.   
2) Load saved model prediction and confidence files from 1.   
3) Visualize confidence values for true and false predictions per class to determine thresholds for use with image type classifiers.

### Imports
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For working with data and plotting graphs
import itertools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# For image classification and training
import tensorflow as tf

%cd drive/My Drive/summer20/classification/image_type/det_conf_threshold/

### 1) Save model predictions and confidence values for 1500 images per class  (Map, Phylogeny, Illustration, Herbarium Sheet, Null) for each model
---   
True and false predictions by confidence value will be used to compare model performance per class. Get values for models from 2 best training runs. 

In [ ]:
# Define functions

# TO DO: Do you want to display classification results for the most recently trained model?
answer = "No" #@param ["Yes", "No"]
# TO DO: If No, manually input desired training attempt number to the right
if answer == "Yes":
  # Display results from most recent training attempt
  last_attempt = !ls /content/drive/'My Drive'/summer20/classification/image_type/saved_models/ | tail -n 1
  TRAIN_SESS_NUM = str(last_attempt.n)
else:
  TRAIN_SESS_NUM = "13" #@param ["11", "13"]

# Load trained model from path
saved_model_path = '/content/drive/My Drive/summer20/classification/image_type/saved_models/' + TRAIN_SESS_NUM
imtype_model = tf.keras.models.load_model(saved_model_path)

# TO DO: Select model type
module_selection = ("mobilenet_v2_1.0_224", 224) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
IMAGE_SIZE = (pixels, pixels)

# Function for plotting classification results with color-coded label if true or false prediction
label_names = ['herb', 'illus', 'maps', 'null', 'phylo']

In [ ]:
# Run inference
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import time

# TO DO: Choose the image class to run for (Run 1x per class per model)
base = '/content/drive/My Drive/summer20/classification/'
classifier = "image_type/" #@param ["flower_fruit/", "image_type/"]
true_imclass = "null" #@param ["maps", "phylo", "illus", "herb", "null"]
PATH_TO_TEST_IMAGES_DIR = base + classifier + "images/" + true_imclass
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]
outpath = base + classifier + 'det_conf_threshold/imagetype_conf_threshold_' + TRAIN_SESS_NUM + "_" + true_imclass + ".csv"

# For determinining confidence threshold
confi = []
true_id = []
det_id = []
colormode = []

# Loops through first 5 image urls from the text file
start = 0 #@param {type:"number"}
end =  1500 #@param {type:"number"}
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[start:end], start=1):
    # Load in image
    imga = Image.open(im_path)
    img = imga.convert('RGB')
    image = img.resize(IMAGE_SIZE)
    image = np.reshape(image,[1,pixels,pixels,3])
    image = image*1./255
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    predictions = imtype_model.predict(image, batch_size=1)
    label_num = np.argmax(predictions)
    conf = predictions[0][label_num]
    otherconfa = predictions[0][:label_num]
    otherconfb = predictions[0][label_num+1:]
    imclass = label_names[label_num]
    other_class = label_names[:label_num]+label_names[label_num+1:]
    end_time = time.time()
    # Display progress message after each image
    print('Inference complete for {} of {} images'.format(im_num, (end-start)))

    # Record confidence, true id, determined id to export and choose confidence thresholds
    confi.append(conf)
    true_id.append(true_imclass)
    det_id.append(imclass.lower())
    colormode.append(imga.getbands())

# Combine to df
imtype_conf = pd.DataFrame(([confi, true_id, det_id, colormode]))
imtype_conf = imtype_conf.transpose()

# TO DO: 
imtype_conf.to_csv(outpath, index=False, header=("confidence", "true_id", "det_id", "colormode"))
print(imtype_conf.head())

### 2) Load saved model prediction and confidence files from 1
---

In [ ]:
%cd drive/My Drive/summer20/classification/image_type/det_conf_threshold/

# Combine confidence threshold values for classes 1-3 for all models
#base = 'imagetype_conf_threshold_11_' 
base = 'imagetype_conf_threshold_13_'
exts = ["maps.csv", "phylo.csv", "illus.csv", "herb.csv", "null.csv"]

# Combine all files in the list
all_filenames = [base + e for e in exts]
mod = pd.concat([pd.read_csv(f, sep=',', header=0, na_filter = False) for f in all_filenames])
print("Model:")
print(len(mod))
print(mod.head())

### 3) Look at prediction error and confidence for each class
---   

In [ ]:
# Trained model used in 2 above
df = mod.copy()

thresh=1.6 #@param

## Split by True or False determined image ID
df['det'] = (df["true_id"] == df["det_id"])
tru = df.loc[df.det, :] # True ID
fal = df.loc[~df.det, :] # False ID

## and by Image class
# Map
map_tru = tru.loc[tru["true_id"] == "maps", :] # and True ID
mt_conf = map_tru['confidence']
map_fal = fal.loc[fal["true_id"] == "maps", :]# and False ID
mf_conf = map_fal['confidence']

# Phylogeny
phylo_tru = tru.loc[tru["true_id"] == "phylo", :] # and True ID
pt_conf = phylo_tru['confidence']
phylo_fal = fal.loc[fal["true_id"] == "phylo", :] # and False ID
pf_conf = phylo_fal['confidence']

# Illustration
illus_tru = tru.loc[tru["true_id"] == "illus", :] # and True ID
it_conf = illus_tru['confidence']
illus_fal = fal.loc[fal["true_id"] == "illus", :] # and False ID
if_conf = illus_fal['confidence']

# Herbarium Sheet
herb_tru = tru.loc[tru["true_id"] == "herb", :] # and True ID
ht_conf = herb_tru['confidence']
herb_fal = fal.loc[fal["true_id"] == "herb", :] # and False ID
hf_conf = herb_fal['confidence']

# None
null_tru = tru.loc[tru["true_id"] == "null", :] # and True ID
nt_conf = null_tru['confidence']
null_fal = fal.loc[fal["true_id"] == "null", :] # and False ID
nf_conf = null_fal['confidence']

## Plot parameters
kwargs = dict(alpha=0.5, bins=15)
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, figsize=(10, 10), constrained_layout=True)
#fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, figsize=(10, 10), constrained_layout=True)
fig.suptitle('Prediction Confidence by Class')

# Map
ax1.hist(mt_conf, color='y', label='True Det', **kwargs)
ax1.hist(mf_conf, color='r', label='False Det', **kwargs)
ax1.set_title("Maps (n=1500 images)")
ax1.legend();

# Phylogeny
ax2.hist(pt_conf, color='y', label='True Det', **kwargs)
ax2.hist(pf_conf, color='r', label='False Det', **kwargs)
ax2.set_title("Phylogeny (n=1024 images)")
ax2.legend();

# Illustration
ax3.hist(it_conf, color='y', label='True Det', **kwargs)
ax3.hist(if_conf, color='r', label='False Det', **kwargs)
ax3.set_title("Illustration (n=1500 images)")
ax3.legend();

# Herbarium Sheet
ax4.hist(ht_conf, color='y', label='True Det', **kwargs)
ax4.hist(hf_conf, color='r', label='False Det', **kwargs)
ax4.set_title("Herbarium Sheet (n=1500 images)")
ax4.legend();

# None
ax5.hist(nt_conf, color='y', label='True Det', **kwargs)
ax5.hist(nf_conf, color='r', label='False Det', **kwargs)
ax5.set_title("None (n=1500 images)")
ax5.legend();

# Y-axis label
for ax in fig.get_axes():
    ax.set(ylabel='Freq (# imgs)')
    if thresh:
      ax.axvline(thresh, color='k', linestyle='dashed', linewidth=1)

# TO DO: Choose model name for exporting graphs
#figname = 'inceptionv3_11' + '.png'
#figname = 'mobilenetv2_13' + '.png'
#fig.savefig(figname)

### 4) Test confidence threshold values
---

In [ ]:
# Trained model used in 2 above
df = mod.copy()

# Split by True or False determined image ID
df['det'] = (df["true_id"] == df["det_id"])
tru = df.loc[df.det, :] # True ID
fal = df.loc[~df.det, :] # False ID
# Export results for further analysis
df[['true_id','det_id']].to_csv('true_false_dets_forbarcharts_inception_pooledillus.tsv', sep='\t', index=0)
  
# Confidence values to test  
conf_vals = [2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.8]
for conf_val in conf_vals: 
  df1 = df.loc[df["confidence"] > conf_val, :]
  new = tru.loc[tru["confidence"] > conf_val, :]
  new_fal = fal.loc[fal["confidence"] > conf_val, :]
  all_vals = new.append(new_fal)
  print("Confidence Value: {}".format(conf_val))
  print("Accuracy for confidence > {}: {}, all classes".format(conf_val, len(new)/len(all_vals)))
  print("Predictions Retained (%): {}".format(len(df1)/len(df)))
  print("True Predictions Retained (%): {}".format(len(new)/len(tru)))
  print("False Predictions Retained (%): {}".format(len(new_fal)/len(fal)))
  print("Accuracy for confidence > {}, by class:".format(conf_val))
  print("Maps: {}".format(len(new.loc[new["true_id"] == "maps", :])/len(all_vals.loc[all_vals["true_id"] == "maps", :])))
  print("Phylo: {}".format(len(new.loc[new["true_id"] == "phylogeny", :])/len(all_vals.loc[all_vals["true_id"] == "phylogeny", :])))
  print("Herb: {}".format(len(new.loc[new["true_id"] == "herbarium sheet", :])/len(all_vals.loc[all_vals["true_id"] == "herbarium sheet", :])))
  print("Null: {}".format(len(new.loc[new["true_id"] == "null", :])/len(all_vals.loc[all_vals["true_id"] == "null", :])))
  print("Illus: {}\n".format(len(new.loc[new["true_id"] == "illustration", :])/len(all_vals.loc[all_vals["true_id"] == "illustration", :])))

### 5) Inspect detections by image colorspace 
--- 
Noticed that many false dets in illustrations were from greyscale color mode ('L' in pillow)

In [ ]:
print("Illus:")
print("False det, True det")
print(len(illus_fal), len(illus_tru))
print("False det in greyscale")
illus_falu_col = illus_fal.loc[illus_fal["colormode"]=="('L',)", :]
print(len(illus_fal_col))
print("True det in greyscale")
illus_tru_col = illus_tru.loc[illus_tru["colormode"]=="('L',)", :]
print(len(illus_tru_col))

print("\nPhylo:")
print(len(phylo_fal), len(phylo_tru))
phylo_fal_col = phylo_fal.loc[phylo_fal["colormode"]=="('L',)", :]
print(len(phylo_fal_col))
phylo_tru_col = phylo_tru.loc[phylo_tru["colormode"]=="('L',)", :]
print(len(phylo_tru_col))

print("\nMaps:")
print(len(map_fal), len(map_tru))
map_fal_col = map_fal.loc[map_fal["colormode"]=="('L',)", :]
print(len(map_fal_col))
map_tru_col = map_tru.loc[map_tru["colormode"]=="('L',)", :]
print(len(map_tru_col))

print("\nHerb:")
print(len(herb_fal), len(herb_tru))
herb_fal_col = herb_fal.loc[herb_fal["colormode"]=="('L',)", :]
print(len(herb_fal_col))
herb_tru_col = herb_tru.loc[herb_tru["colormode"]=="('L',)", :]
print(len(herb_tru_col))

print("\nNone:")
print(len(null_fal), len(null_tru))
null_fal_col = null_fal.loc[null_fal["colormode"]=="('L',)", :]
print(len(null_fal_col))
null_tru_col = null_tru.loc[null_tru["colormode"]=="('L',)", :]
print(len(null_tru_col))